In [48]:
import torch
from torch.autograd import Variable
import torch.nn

In [49]:
#import the data
from google.colab import files
uploaded = files.upload()

Saving auto-mpg.csv to auto-mpg (1).csv


In [50]:
import pandas as pd
import numpy as np

In [75]:
data = pd.read_csv("auto-mpg.csv")

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [77]:
target = data['mpg']
data.drop(['mpg'],inplace = True,axis =1) 

In [78]:
#seperate numerical and categorical columns
from sklearn.compose import make_column_selector 
numeric_cols = make_column_selector(dtype_exclude=object) #calling the function and excluding the object col's
data_numeric = numeric_cols(data) # selecting the numeric columns
numeric = data[data_numeric] # selecting the numeric data by using numeric col's


In [79]:
#scaling to get the same range of values in the dataset
from sklearn.preprocessing import StandardScaler
scaling = StandardScaler() #calling the function
scaling.fit(numeric) # fitting the data to apply scaling
numeric_scaling = scaling.transform(numeric) # it will take every value and apply the formula of standard scaler


In [80]:
numeric_scaling_df = pd.DataFrame(numeric_scaling,columns = data_numeric) #we are converting the numpy array to dataframe
numeric_scaling_df.head()

,cylinders,displacement,weight,acceleration,model year,origin
0,1.498191,1.090604,0.630870,-1.295498,-1.627426,-0.715145
1,1.498191,1.503514,0.854333,-1.477038,-1.627426,-0.715145
2,1.498191,1.196232,0.550470,-1.658577,-1.627426,-0.715145
3,1.498191,1.061796,0.546923,-1.295498,-1.627426,-0.715145
4,1.498191,1.042591,0.565841,-1.840117,-1.627426,-0.715145


In [81]:
data.drop(['car name'],inplace = True, axis = 1)

In [82]:
horsepower_nomissing = data[data['horsepower']!='?']['horsepower']
horsepower_nomissing = horsepower_nomissing.astype(int)
np.mean(horsepower_nomissing)
data['horsepower'].replace('?',np.mean(horsepower_nomissing),inplace = True)

In [83]:
data['horsepower'] = data['horsepower'].astype(int)

In [84]:
#final data preparation
final_data = pd.concat([numeric_scaling_df,data['horsepower']],axis = 1)
x = final_data
y = target

In [85]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(x_test.shape)


(318, 7)
(80, 7)
(318,)
(80, 7)


In [86]:
# converting to numpy arrays and then to tensors
x_train1 = x_train.to_numpy()
x_test1 = x_test.to_numpy()
y_train1 = y_train.to_numpy()
y_test1 = y_test.to_numpy()


In [87]:
#numpy arrays to tensor
x_train_tensor = torch.from_numpy(x_train1)
x_test_tensor = torch.from_numpy(x_test1)
y_train_tensor = torch.from_numpy(y_train1)
y_test_tensor = torch.from_numpy(y_test1)


In [88]:
print(x_train_tensor.shape)
print(x_test_tensor.shape)
print(y_train_tensor.shape)
print(y_test_tensor.shape)

torch.Size([318, 7])
torch.Size([80, 7])
torch.Size([318])
torch.Size([80])


In [94]:
#pytorch Linear regression
class LinearRegressionModel(torch.nn.Module):

  def __init__(self):
    super(LinearRegressionModel, self).__init__()
    self.linear = torch.nn.Linear(7,1)

  def forward(self,x):
    y_pred = self.linear(x)
    return y_pred

our_model = LinearRegressionModel()

criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(our_model.parameters(), lr = 0.01)

for epoch in range(2):
  pred_y = our_model(x_train_tensor.float())
  #training loss
  loss = criterion(pred_y, y_train_tensor.float())
  print("training loss\n",loss)
  

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  

  #print('epoch {}, loss {}'.format(epoch,loss.item()))

#testing
our_model.eval()
pred_y_test = our_model(x_test_tensor.float())
test_loss = criterion(pred_y_test,y_test_tensor.float())
print("testing loss\n",test_loss)







training loss
 tensor(46848096., grad_fn=<MseLossBackward0>)
training loss
 tensor(2.1460e+22, grad_fn=<MseLossBackward0>)
testing loss
 tensor(8.8128e+35, grad_fn=<MseLossBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([318])) that is different to the input size (torch.Size([318, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([80])) that is different to the input size (torch.Size([80, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [90]:
pred_y

tensor([[1.3296e+08],
        [3.1011e+08],
        [1.8610e+08],
        [2.4809e+08],
        [1.8612e+08],
        [1.3828e+08],
        [1.3297e+08],
        [1.1527e+08],
        [1.6841e+08],
        [2.6582e+08],
        [1.1879e+08],
        [1.9497e+08],
        [3.3669e+08],
        [1.2762e+08],
        [1.4714e+08],
        [1.2413e+08],
        [2.6582e+08],
        [9.3993e+07],
        [1.5066e+08],
        [1.7370e+08],
        [1.8609e+08],
        [1.3298e+08],
        [2.7468e+08],
        [1.5423e+08],
        [1.1527e+08],
        [2.4810e+08],
        [1.7724e+08],
        [1.1526e+08],
        [1.4891e+08],
        [2.4810e+08],
        [8.1586e+07],
        [1.3296e+08],
        [1.9496e+08],
        [2.8352e+08],
        [1.2411e+08],
        [1.5955e+08],
        [1.4714e+08],
        [1.6838e+08],
        [1.2588e+08],
        [3.1010e+08],
        [3.9872e+08],
        [1.4006e+08],
        [1.6841e+08],
        [2.0561e+08],
        [1.2764e+08],
        [1